In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
buoy = pd.read_csv("/Users/matthewq/Geol_599/week9/Waves_2023.txt", sep='\s+') 
buoy.describe()

,YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
count,17285.0,17285.000000,17285.000000,17285.000000,17285.000000,17285.0,17285.0,17285.0,17285.000000,17285.000000,17285.000000,17285.000000,17285.0,17285.0,17285.000000,17285.0,17285.0,17285.0
mean,2023.0,6.526063,15.800579,11.499277,40.985594,999.0,99.0,99.0,1.083535,12.161879,6.813054,234.556205,9999.0,999.0,17.196535,999.0,99.0,99.0
std,0.0,3.467253,8.817920,6.925343,15.007646,0.0,0.0,0.0,2.296129,4.165737,2.803208,49.429981,0.0,0.0,2.564664,0.0,0.0,0.0
min,2023.0,1.000000,1.000000,0.000000,0.000000,999.0,99.0,99.0,0.330000,2.860000,3.420000,10.000000,9999.0,999.0,11.800000,999.0,99.0,99.0
25%,2023.0,4.000000,8.000000,5.000000,26.000000,999.0,99.0,99.0,0.730000,9.090000,5.500000,179.000000,9999.0,999.0,14.600000,999.0,99.0,99.0
50%,2023.0,6.000000,16.000000,11.000000,26.000000,999.0,99.0,99.0,0.880000,12.500000,6.360000,263.000000,9999.0,999.0,17.500000,999.0,99.0,99.0
75%,2023.0,10.000000,23.000000,18.000000,56.000000,999.0,99.0,99.0,1.140000,14.290000,7.550000,271.000000,9999.0,999.0,19.400000,999.0,99.0,99.0
max,2023.0,12.000000,31.000000,23.000000,56.000000,999.0,99.0,99.0,99.000000,99.000000,99.000000,999.000000,9999.0,999.0,23.900000,999.0,99.0,99.0


In [3]:
buoy_use = buoy[['WVHT', 'DPD', 'APD', 'MWD', 'WTMP']].copy()
buoy_use_na = buoy_use.replace(99,np.NaN) #convert any 99 values to NaN
buoy_use_na = buoy_use_na.replace(999,np.NaN) #convert any 999 values to NaN
buoy_use_nona = buoy_use_na.dropna() #drop any NaN values, which now contains 99 and 999 values
buoy_use_nona.describe()

,WVHT,DPD,APD,MWD,WTMP
count,17276.000000,17276.000000,17276.000000,17276.000000,17276.000000
mean,1.032525,12.116640,6.765029,234.157965,17.195294
std,0.526637,3.664928,1.852623,46.260000,2.564729
min,0.330000,2.860000,3.420000,10.000000,11.800000
25%,0.730000,9.090000,5.500000,179.000000,14.600000
50%,0.880000,12.500000,6.360000,263.000000,17.500000
75%,1.140000,14.290000,7.550000,271.000000,19.400000
max,5.130000,25.000000,15.790000,306.000000,23.900000


In [4]:
X = buoy_use[["DPD", "APD", "MWD", 'WTMP']].to_numpy()
y = buoy_use['WVHT']


#Splitting data set - test = 10% of data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

#splitting training data set further to get validation set (15% of whole data set) leaving training set 75% of whole data set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.17)

In [5]:
nreg = MLPRegressor(hidden_layer_sizes=(16,16,16)).fit(X_train, y_train)

In [6]:
nreg_ypred_test = nreg.predict(X_test)
nreg_ypred_train = nreg.predict(X_train)

nreg_mse_test = mean_squared_error(nreg_ypred_test, y_test)
nreg_mse_train = mean_squared_error(nreg_ypred_train, y_train)

print(nreg_mse_train, nreg_mse_test)

0.2027282542325785 0.1854118097208641


In [7]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

reg_ypred_test = reg.predict(X_test)
reg_ypred_train = reg.predict(X_train)

reg_mse_train = mean_squared_error(reg_ypred_train, y_train)
reg_mse_test = mean_squared_error(reg_ypred_test, y_test)

print(reg_mse_train, reg_mse_test)

1.9219447324076286 1.305445604474636


1- 

When compared to a baseline linear regression model, the MLPRegressor model works better as the mean squared error is close to an order of magnitude lower.

2- 

MLPRegressor optimizes the squared error using LBFGS or stochastic gradient descent. The default cost function ('adam') is a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba 

In [8]:
#different neural network architectures
nreg1 = MLPRegressor(hidden_layer_sizes=(1,)).fit(X_train, y_train)
nreg2 = MLPRegressor(hidden_layer_sizes=(16,)).fit(X_train, y_train)
nreg3 = MLPRegressor(hidden_layer_sizes=(1, 16,)).fit(X_train, y_train)
nreg4 = MLPRegressor(hidden_layer_sizes=(16, 16,)).fit(X_train, y_train)
nreg5 = MLPRegressor(hidden_layer_sizes=(16, 4, 16,)).fit(X_train, y_train)
nreg6 = MLPRegressor(hidden_layer_sizes=(16, 16, 16,)).fit(X_train, y_train)
nreg7 = MLPRegressor(hidden_layer_sizes=(16, 16, 16, 16,)).fit(X_train, y_train)
nreg8 = MLPRegressor(hidden_layer_sizes=(16, 32, 64, 128,)).fit(X_train, y_train)
nreg9 = MLPRegressor(hidden_layer_sizes=(1, 1, 1, 1,)).fit(X_train, y_train)
nreg10 = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100,)).fit(X_train, y_train)

/Users/matthewq/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
nreg1_ypred_test = nreg1.predict(X_test)
nreg1_ypred_train = nreg1.predict(X_train)

nreg1_mse_train = mean_squared_error(nreg1_ypred_train, y_train)
nreg1_mse_test = mean_squared_error( nreg1_ypred_test, y_test)

print(nreg1_mse_train, nreg1_mse_test)

0.24032346191365375 0.23350166685897092


In [10]:
nreg2_ypred_test = nreg2.predict(X_test)
nreg2_ypred_train = nreg2.predict(X_train)

nreg2_mse_train = mean_squared_error(nreg2_ypred_train, y_train)
nreg2_mse_test = mean_squared_error(nreg2_ypred_test, y_test)

print(nreg2_mse_train, nreg2_mse_test)

0.1911908775801144 0.18302634236881962


In [11]:
nreg3_ypred_test = nreg3.predict(X_test)
nreg3_ypred_train = nreg3.predict(X_train)

nreg3_mse_train = mean_squared_error(nreg3_ypred_train, y_train)
nreg3_mse_test = mean_squared_error(nreg3_ypred_test, y_test)

print(nreg3_mse_train, nreg3_mse_test)

0.21461304612798118 0.20421193540368116


In [12]:
nreg4_ypred_test = nreg4.predict(X_test)
nreg4_ypred_train = nreg4.predict(X_train)

nreg4_mse_test = mean_squared_error(nreg4_ypred_test, y_test)
nreg4_mse_train = mean_squared_error(nreg4_ypred_train, y_train)

print(nreg4_mse_train, nreg4_mse_test)

0.20787244728533188 0.21164902571998695


In [13]:
nreg5_ypred_test = nreg5.predict(X_test)
nreg5_ypred_train = nreg5.predict(X_train)

nreg5_mse_test = mean_squared_error(nreg5_ypred_test, y_test)
nreg5_mse_train = mean_squared_error(nreg5_ypred_train, y_train)

print(nreg5_mse_train, nreg5_mse_test)

6.220340760445386 0.28312433011354704


In [14]:
nreg6_ypred_test = nreg6.predict(X_test)
nreg6_ypred_train = nreg6.predict(X_train)

nreg6_mse_test = mean_squared_error(nreg6_ypred_test, y_test)
nreg6_mse_train = mean_squared_error(nreg6_ypred_train, y_train)

print(nreg6_mse_train, nreg6_mse_test)

0.19089163410676774 0.18029171784282702


In [15]:
nreg7_ypred_test = nreg7.predict(X_test)
nreg7_ypred_train = nreg7.predict(X_train)

nreg7_mse_test = mean_squared_error(nreg7_ypred_test, y_test)
nreg7_mse_train = mean_squared_error(nreg7_ypred_train, y_train)

print(nreg7_mse_train, nreg7_mse_test)

0.20835464980929924 0.19348918265759069


In [16]:
nreg8_ypred_test = nreg8.predict(X_test)
nreg8_ypred_train = nreg8.predict(X_train)

nreg8_mse_test = mean_squared_error(nreg8_ypred_test, y_test)
nreg8_mse_train = mean_squared_error(nreg8_ypred_train, y_train)

print(nreg8_mse_train, nreg8_mse_test)

0.20525675046747063 0.18835291657143238


In [17]:
nreg9_ypred_test = nreg9.predict(X_test)
nreg9_ypred_train = nreg9.predict(X_train)

nreg9_mse_train = mean_squared_error(nreg9_ypred_train, y_train)
nreg9_mse_test = mean_squared_error(nreg9_ypred_test, y_test)

print(nreg9_mse_train, nreg9_mse_test)

6.220309856431839 0.28437739068840695


In [18]:
nreg10_ypred_test = nreg10.predict(X_test)
nreg10_ypred_train = nreg10.predict(X_train)

nreg10_mse_test = mean_squared_error(nreg10_ypred_test, y_test)
nreg10_mse_train = mean_squared_error(nreg10_ypred_train, y_train)

print(nreg10_mse_train, nreg10_mse_test)

0.19206964833476906 0.17971015982255217


In [19]:
error = pd.DataFrame(columns=['Layers', 'Nodes', 'Train MSE', 'Test MSE'])

In [20]:
row1 = ['1', '1', nreg1_mse_train, nreg1_mse_test]
row2 = ['1', '16', nreg2_mse_train, nreg2_mse_test]
row3 = ['2', '[1, 16]', nreg3_mse_train, nreg3_mse_test]
row4 = ['2', '[16, 16]', nreg4_mse_train, nreg4_mse_test]
row5 = ['3', '[16, 4, 16]', nreg5_mse_train, nreg5_mse_test]
row6 = ['3', '[16, 16, 16]', nreg6_mse_train, nreg6_mse_test]
row7 = ['4', '[16, 16, 16, 16]', nreg7_mse_train, nreg7_mse_test]
row8 = ['4', '[16, 32, 64, 128]', nreg8_mse_train, nreg8_mse_test]
row9 = ['5', '[1, 1, 1, 1, 1]', nreg9_mse_train, nreg9_mse_test]
row10 = ['5', '[100, 100, 100, 100, 100]', nreg10_mse_train, nreg10_mse_test]

In [21]:
error.loc[len(error)] = row1

In [22]:
error.loc[len(error)] = row2

In [23]:
error.loc[len(error)] = row3

In [24]:
error.loc[len(error)] = row4

In [25]:
error.loc[len(error)] = row5

In [26]:
error.loc[len(error)] = row6

In [27]:
error.loc[len(error)] = row7

In [28]:
error.loc[len(error)] = row8

In [29]:
error.loc[len(error)] = row9

In [30]:
error.loc[len(error)] = row10

In [31]:
#3- 
print(error)
#all the training errors are higher than the test errors. Seems a bit odd

  Layers                      Nodes  Train MSE  Test MSE
0      1                          1   0.240323  0.233502
1      1                         16   0.191191  0.183026
2      2                    [1, 16]   0.214613  0.204212
3      2                   [16, 16]   0.207872  0.211649
4      3                [16, 4, 16]   6.220341  0.283124
5      3               [16, 16, 16]   0.190892  0.180292
6      4           [16, 16, 16, 16]   0.208355  0.193489
7      4          [16, 32, 64, 128]   0.205257  0.188353
8      5            [1, 1, 1, 1, 1]   6.220310  0.284377
9      5  [100, 100, 100, 100, 100]   0.192070  0.179710


4- 
Overall id doesnt really seem like changing the number of hidden layers and nodes has had a significant effect on the performance of the model. There are two training errors that are really high relative to the rest ([16,4,6] and [1,1,1,1]). I dont really understand what is causing this.

In [35]:
nreg_id = MLPRegressor(hidden_layer_sizes=(100,100,100,100), activation = 'identity', max_iter=300).fit(X_train, y_train)

nreg_id_ypred_test = nreg_id.predict(X_test)
nreg_id_ypred_train = nreg_id.predict(X_train)

nreg_id_mse_test = mean_squared_error(nreg_id_ypred_test, y_test)
nreg_id_mse_train = mean_squared_error(nreg_id_ypred_train, y_train)

print(nreg_id_mse_train, nreg_id_mse_test)

3.1274462208772795 2.006133032437981


In [34]:
nreg_id = MLPRegressor(hidden_layer_sizes=(1,1,1,1), activation = 'identity', max_iter=300).fit(X_train, y_train)

nreg_id_ypred_test = nreg_id.predict(X_test)
nreg_id_ypred_train = nreg_id.predict(X_train)

nreg_id_mse_test = mean_squared_error(nreg_id_ypred_test, y_test)
nreg_id_mse_train = mean_squared_error(nreg_id_ypred_train, y_train)

print(nreg_id_mse_train, nreg_id_mse_test)

1.9423958994454145 1.280563611690215


In [36]:
nreg_log = MLPRegressor(hidden_layer_sizes=(100,100,100,100), activation = 'logistic').fit(X_train, y_train)

nreg_log_ypred_test = nreg_log.predict(X_test)
nreg_log_ypred_train = nreg_log.predict(X_train)

nreg_log_mse_test = mean_squared_error(nreg_log_ypred_test, y_test)
nreg_log_mse_train = mean_squared_error(nreg_log_ypred_train, y_train)

print(nreg_log_mse_train, nreg_log_mse_test)

0.18158743490003743 0.16555865830589842


In [37]:
nreg_log = MLPRegressor(hidden_layer_sizes=(1,1,1,1), activation = 'logistic').fit(X_train, y_train)

nreg_log_ypred_test = nreg_log.predict(X_test)
nreg_log_ypred_train = nreg_log.predict(X_train)

nreg_log_mse_test = mean_squared_error(nreg_log_ypred_test, y_test)
nreg_log_mse_train = mean_squared_error(nreg_log_ypred_train, y_train)

print(nreg_log_mse_train, nreg_log_mse_test)

6.220303318162661 0.28409845640770365


In [38]:
nreg_tanh = MLPRegressor(hidden_layer_sizes=(100,100,100,100), activation = 'tanh').fit(X_train, y_train)

nreg_tanh_ypred_test = nreg_tanh.predict(X_test)
nreg_tanh_ypred_train = nreg_tanh.predict(X_train)

nreg_tanh_mse_test = mean_squared_error(nreg_tanh_ypred_test, y_test)
nreg_tanh_mse_train = mean_squared_error(nreg_tanh_ypred_train, y_train)

print(nreg_tanh_mse_train, nreg_tanh_mse_test)

6.2295075723692355 0.28092100574625506


In [39]:
nreg_tanh = MLPRegressor(hidden_layer_sizes=(1,1,1,1), activation = 'tanh').fit(X_train, y_train)

nreg_tanh_ypred_test = nreg_tanh.predict(X_test)
nreg_tanh_ypred_train = nreg_tanh.predict(X_train)

nreg_tanh_mse_test = mean_squared_error(nreg_tanh_ypred_test, y_test)
nreg_tanh_mse_train = mean_squared_error(nreg_tanh_ypred_train, y_train)

print(nreg_tanh_mse_train, nreg_tanh_mse_test)

6.220309735557572 0.28437420030626187


5- 

Changing the activation function from the default ('relu') to identity does have a big effect on the performance of the neural network model. For the 4 hidden layers of 100 nodes, the test and train MSE are significantly worse using an identity activation function. 

Changing to a logistic activation function has little effect. 

Changing to a tanh activation funciton did not affect the 1,1,1,1 neural network but did worsen the performance of the 100,100,100,100 network model.